In [1]:
import ee
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import time
import utils_ee_Export_Phen

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
#read the csv file
dfY = pd.read_csv('/Users/samzhang/Library/CloudStorage/GoogleDrive-chishan2@illinois.edu/My Drive/DomainAdapation/GEE/USID2.csv')
dfY

,GEOID
0,17001
1,17003
2,17005
3,17007
4,17009
...,...
782,55131
783,55133
784,55135
785,55137


In [4]:
#set '0' as string
dfY['GEOID'] = dfY['GEOID'].astype(str)

## select the counties as featurColl

In [5]:
USCounties = ee.FeatureCollection('TIGER/2018/Counties')
# select the US counties by the 0 column in the dfY
countyCol = USCounties.filter(ee.Filter.inList('GEOID',dfY['GEOID'].tolist()))

In [16]:
# geemap.ee_to_shp(countyCol, filename='../data/temp/USCounties.shp')
#read the shapefile
# gdf = gpd.read_file('../data/temp/USCounties.shp')
# gdf

,AWATER,GEOID,LSAD,CBSAFP,STATEFP,INTPTLAT,FUNCSTAT,CSAFP,COUNTYFP,CLASSFP,NAME,NAMELSAD,COUNTYNS,INTPTLON,ALAND,MTFCC,METDIVFP,geometry
0,9301714,17121,06,16460,17,+38.6481057,A,476,121,H1,Marion,Marion County,00424259,-088.9204885,1482641176,G4020,NaN,"POLYGON ((-89.14441 38.47387, -89.14432 38.474..."
1,6462629,17005,06,41180,17,+38.8859240,A,476,005,H1,Bond,Bond County,00424204,-089.4365916,985073312,G4020,NaN,"POLYGON ((-89.63929 38.99900, -89.63924 38.999..."
2,20311971,17083,06,41180,17,+39.0801918,A,476,083,H1,Jersey,Jersey County,00424243,-090.3613647,956445338,G4020,NaN,"POLYGON ((-90.60356 39.11749, -90.60356 39.117..."
3,42001352,17163,06,41180,17,+38.4701976,A,476,163,H1,St. Clair,St. Clair County,01784987,-089.9285459,1703509128,G4020,NaN,"POLYGON ((-90.26411 38.52072, -90.26406 38.521..."
4,75635324,17027,06,41180,17,+38.6062953,A,476,027,H1,Clinton,Clinton County,00424215,-089.4262278,1227664369,G4020,NaN,"POLYGON ((-89.70693 38.65501, -89.70266 38.655..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,392038136,55083,06,24580,55,+44.9965754,A,267,083,H1,Oconto,Oconto County,01581102,-088.2065162,2583464883,G4020,NaN,"POLYGON ((-88.68009 45.20501, -88.67991 45.214..."
954,221808928,55009,06,24580,55,+44.4740244,A,267,009,H1,Brown,Brown County,01581064,-087.9961303,1372933176,G4020,NaN,"POLYGON ((-88.25257 44.67965, -88.25257 44.679..."
955,41883197,55115,06,43020,55,+44.7896407,A,267,115,H1,Shawano,Shawano County,01581117,-088.7558134,2313441606,G4020,NaN,"POLYGON ((-89.22439 44.79797, -89.22439 44.798..."
956,17006539,18097,06,26900,18,+39.7829759,C,294,097,H6,Marion,Marion County,00450371,-086.1357939,1026784675,G4020,NaN,"POLYGON ((-86.32811 39.86561, -86.32811 39.866..."


## map the modis by soybean map and pheno stages

In [6]:
#calculate evi2
def addevi(image):
    evi = image.expression(
        '2.5 * float(nir - red)*0.0001 / float(nir*0.0001 + 2.4 * red*0.0001 + 1)',
        {
            'red': image.select('Nadir_Reflectance_Band1'),    # 620-670nm, RED
            'nir': image.select('Nadir_Reflectance_Band2'),    # 841-876nm, NIR
            #blue: image.select('Nadir_Reflectance_Band3')    # 459-479nm, BLUE
        });
    return image.addBands(evi.rename(['EVI2']))

In [12]:
# add a scolling bar for for loop
from tqdm import tqdm
from time import sleep

Country='US'

for  yearInt in tqdm(range(2008,2019)):
  year=str(yearInt)
  #read the soybean map
  SoyMap=ee.Image('projects/ee-zcs/assets/CDL_500_MODIS/CDL_soybean_CB_'+year)

  #read the variable
  varibaleList=['tmax','tmean','ppt']
  varName=varibaleList[1]
  Satellite = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').filterDate(str(yearInt-1)+'-1-1',year+'-12-31').select(varName)
  
  # varName='EVI2'
  # Satellite = ee.ImageCollection('MODIS/061/MCD43A4').filterDate(str(yearInt-1)+'-1-1',year+'-12-31').map(addevi).select(varName)

  utils_ee_Export_Phen.main(Satellite=Satellite, countyCol=countyCol, SoyMap=SoyMap, Country=Country,year=year, varName=varName,reducer='mean')


  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:14<00:00,  1.36s/it]


In [10]:
'mean'=='mean'

True

## Read the data

In [27]:
# read geojosn file
pdts=gpd.read_file('/Users/samzhang/Library/CloudStorage/GoogleDrive-zcsbelief@gmail.com/My Drive/GEE_exports/ArgEVI2Hist.geojson')
pdts=pd.DataFrame(pdts)
pdts

,id,ADM0_ES,ADM0_PCODE,ADM1_ES,ADM1_PCODE,ADM2_ES,ADM2_PCODE,ADM2_REF,p1,p10,p20,p30,p40,p50,p60,p70,p80,p90,p99,geometry
0,00000000000000000047,Argentina,AR,Chaco,AR022,O'Higgins,AR022112,O'Higgins,0.360899,0.427825,0.458886,0.482480,0.505538,0.529092,0.552809,0.579890,0.611446,0.661768,0.764238,MULTIPOINT EMPTY
1,0000000000000000004e,Argentina,AR,Chaco,AR022,Independencia,AR022070,Independencia,0.412354,0.478526,0.517648,0.544978,0.576255,0.607134,0.642330,0.685465,0.736377,0.791115,0.877448,MULTIPOINT EMPTY
2,00000000000000000079,Argentina,AR,Chaco,AR022,Maipú,AR022091,Maipu,0.380825,0.435798,0.466938,0.486364,0.505645,0.525381,0.552644,0.580344,0.619099,0.669821,0.794468,MULTIPOINT EMPTY
3,00000000000000000083,Argentina,AR,Chaco,AR022,9 de Julio,AR022105,9 de Julio,0.365184,0.427592,0.462822,0.490256,0.517534,0.552754,0.591600,0.634724,0.693502,0.760117,0.872463,MULTIPOINT EMPTY
4,0000000000000000009d,Argentina,AR,Chaco,AR022,General Belgrano,AR022049,General Belgrano,0.381010,0.459376,0.502105,0.541123,0.579646,0.619387,0.661773,0.704860,0.755911,0.822133,0.888554,MULTIPOINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,000000000000000000d5,Argentina,AR,Santiago del Estero,AR086,Robles,AR086161,Robles,0.324984,0.379784,0.411098,0.432577,0.450206,0.469628,0.485247,0.506866,0.532137,0.574978,0.684040,MULTIPOINT EMPTY
201,00000000000000000141,Argentina,AR,Santiago del Estero,AR086,Aguirre,AR086007,Aguirre,0.294754,0.361308,0.392741,0.419883,0.447425,0.474231,0.501897,0.540816,0.580311,0.634976,0.747559,MULTIPOINT EMPTY
202,000000000000000001ac,Argentina,AR,Santiago del Estero,AR086,Banda,AR086035,Banda,0.329809,0.384957,0.412057,0.431161,0.450905,0.466666,0.486443,0.509909,0.544782,0.607545,0.794598,MULTIPOINT EMPTY
203,000000000000000001cf,Argentina,AR,Santiago del Estero,AR086,Copo,AR086056,Copo,0.409598,0.450211,0.477585,0.503214,0.534263,0.561051,0.590612,0.629841,0.678625,0.748830,0.846908,MULTIPOINT EMPTY


In [16]:
# statistics of the pure pixels in us counties
for  yearInt in range(2008,2019):
    
    year=str(yearInt)
    print(year)
    
    SoyMap=ee.Image('projects/ee-zcs/assets/CDL_500_MODIS/CDL_soybean_CB_'+year)

    VCount = SoyMap.gte(0.50).reduceRegions(**{
        'reducer': ee.Reducer.sum(),
        'collection': countyCol,
        'scale': 500})

    #set the geometry of the fearure collection as null
    def setGeometry(feature):
        return feature.setGeometry(None)
    
    #apply the function to the feature collection
    VCount=VCount.map(setGeometry)

    #convert the feature collection to geopandas
    gdf=geemap.ee_to_geojson(VCount,filename='../data/temp/VCountUS'+year+'.geojson')

    # break


2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [18]:
#read the geojosn file
VCount=gpd.read_file('../data/temp/VCountUS2009.geojson')
VCount

,id,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,sum,geometry
0,00000000000000000003,1482641176,9301714,16460,H1,121,00424259,476,A,17121,+38.6481057,-088.9204885,06,,G4020,Marion,Marion County,17,1520.129412,None
1,00000000000000000004,985073312,6462629,41180,H1,005,00424204,476,A,17005,+38.8859240,-089.4365916,06,,G4020,Bond,Bond County,17,1054.015686,None
2,00000000000000000005,956445338,20311971,41180,H1,083,00424243,476,A,17083,+39.0801918,-090.3613647,06,,G4020,Jersey,Jersey County,17,630.376471,None
3,00000000000000000006,1703509128,42001352,41180,H1,163,01784987,476,A,17163,+38.4701976,-089.9285459,06,,G4020,St. Clair,St. Clair County,17,1582.698039,None
4,00000000000000000007,1227664369,75635324,41180,H1,027,00424215,476,A,17027,+38.6062953,-089.4262278,06,,G4020,Clinton,Clinton County,17,1389.984314,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,000000000000000008ac,2583464883,392038136,24580,H1,083,01581102,267,A,55083,+44.9965754,-088.2065162,06,,G4020,Oconto,Oconto County,55,90.000000,None
954,000000000000000008ad,1372933176,221808928,24580,H1,009,01581064,267,A,55009,+44.4740244,-087.9961303,06,,G4020,Brown,Brown County,55,113.643137,None
955,000000000000000008ae,2313441606,41883197,43020,H1,115,01581117,267,A,55115,+44.7896407,-088.7558134,06,,G4020,Shawano,Shawano County,55,138.301961,None
956,0000000000000000060f,1026784675,17006539,26900,H6,097,00450371,294,C,18097,+39.7829759,-086.1357939,06,,G4020,Marion,Marion County,18,112.784314,None


In [19]:
VCount['sum']

0      1520.129412
1      1054.015686
2       630.376471
3      1582.698039
4      1389.984314
          ...     
953      90.000000
954     113.643137
955     138.301961
956     112.784314
957      67.066667
Name: sum, Length: 958, dtype: float64

In [21]:
VCount['values'+str(year)]=dataT['sum']

In [22]:
VCount

,id,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,...,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP,sum,geometry,values2008
0,00000000000000000003,1482641176,9301714,16460,H1,121,00424259,476,A,17121,...,-088.9204885,06,,G4020,Marion,Marion County,17,1520.129412,None,1274.921569
1,00000000000000000004,985073312,6462629,41180,H1,005,00424204,476,A,17005,...,-089.4365916,06,,G4020,Bond,Bond County,17,1054.015686,None,1031.294118
2,00000000000000000005,956445338,20311971,41180,H1,083,00424243,476,A,17083,...,-090.3613647,06,,G4020,Jersey,Jersey County,17,630.376471,None,650.168627
3,00000000000000000006,1703509128,42001352,41180,H1,163,01784987,476,A,17163,...,-089.9285459,06,,G4020,St. Clair,St. Clair County,17,1582.698039,None,1173.945098
4,00000000000000000007,1227664369,75635324,41180,H1,027,00424215,476,A,17027,...,-089.4262278,06,,G4020,Clinton,Clinton County,17,1389.984314,None,1071.752941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,000000000000000008ac,2583464883,392038136,24580,H1,083,01581102,267,A,55083,...,-088.2065162,06,,G4020,Oconto,Oconto County,55,90.000000,None,133.294118
954,000000000000000008ad,1372933176,221808928,24580,H1,009,01581064,267,A,55009,...,-087.9961303,06,,G4020,Brown,Brown County,55,113.643137,None,132.521569
955,000000000000000008ae,2313441606,41883197,43020,H1,115,01581117,267,A,55115,...,-088.7558134,06,,G4020,Shawano,Shawano County,55,138.301961,None,136.658824
956,0000000000000000060f,1026784675,17006539,26900,H6,097,00450371,294,C,18097,...,-086.1357939,06,,G4020,Marion,Marion County,18,112.784314,None,117.992157


In [23]:
for year in range(2008,2019):
    
    dataT=gpd.read_file('../data/temp/VCountUS'+str(year)+'.geojson')

    VCount['values'+str(year)]=dataT['sum']
    print(year)

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [25]:
VCount

,id,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,...,values2009,values2010,values2011,values2012,values2013,values2014,values2015,values2016,values2017,values2018
0,00000000000000000003,1482641176,9301714,16460,H1,121,00424259,476,A,17121,...,1520.129412,1258.992157,1414.627451,1083.960784,1131.909804,1126.207843,1223.619608,1520.180392,1567.152941,1646.737255
1,00000000000000000004,985073312,6462629,41180,H1,005,00424204,476,A,17005,...,1054.015686,1119.976471,1008.647059,943.776471,953.278431,897.843137,970.313725,1076.541176,1245.325490,1234.235294
2,00000000000000000005,956445338,20311971,41180,H1,083,00424243,476,A,17083,...,630.376471,654.121569,607.474510,586.447059,638.270588,677.349020,748.470588,744.050980,794.482353,910.552941
3,00000000000000000006,1703509128,42001352,41180,H1,163,01784987,476,A,17163,...,1582.698039,1389.772549,1399.176471,1121.360784,1275.831373,1193.313725,1376.082353,1356.490196,1708.039216,1510.494118
4,00000000000000000007,1227664369,75635324,41180,H1,027,00424215,476,A,17027,...,1389.984314,1342.309804,1153.725490,929.160784,1219.937255,982.466667,999.949020,1227.074510,1461.415686,1334.643137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,000000000000000008ac,2583464883,392038136,24580,H1,083,01581102,267,A,55083,...,90.000000,155.411765,126.000000,138.129412,128.086275,218.345098,155.243137,213.301961,182.670588,271.819608
954,000000000000000008ad,1372933176,221808928,24580,H1,009,01581064,267,A,55009,...,113.643137,115.831373,134.352941,105.349020,86.278431,176.776471,159.462745,181.658824,218.729412,234.466667
955,000000000000000008ae,2313441606,41883197,43020,H1,115,01581117,267,A,55115,...,138.301961,141.890196,131.901961,144.094118,133.254902,200.929412,186.854902,218.243137,310.749020,291.694118
956,0000000000000000060f,1026784675,17006539,26900,H6,097,00450371,294,C,18097,...,112.784314,88.470588,101.360784,73.113725,83.611765,89.392157,124.564706,90.145098,153.486275,122.980392


In [26]:
# drop rows with values2008 < 50 of data df
for i in range(2008,2019):
    VCount=VCount[VCount['values'+str(i)]>50]
VCount

,id,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,...,values2009,values2010,values2011,values2012,values2013,values2014,values2015,values2016,values2017,values2018
0,00000000000000000003,1482641176,9301714,16460,H1,121,00424259,476,A,17121,...,1520.129412,1258.992157,1414.627451,1083.960784,1131.909804,1126.207843,1223.619608,1520.180392,1567.152941,1646.737255
1,00000000000000000004,985073312,6462629,41180,H1,005,00424204,476,A,17005,...,1054.015686,1119.976471,1008.647059,943.776471,953.278431,897.843137,970.313725,1076.541176,1245.325490,1234.235294
2,00000000000000000005,956445338,20311971,41180,H1,083,00424243,476,A,17083,...,630.376471,654.121569,607.474510,586.447059,638.270588,677.349020,748.470588,744.050980,794.482353,910.552941
3,00000000000000000006,1703509128,42001352,41180,H1,163,01784987,476,A,17163,...,1582.698039,1389.772549,1399.176471,1121.360784,1275.831373,1193.313725,1376.082353,1356.490196,1708.039216,1510.494118
4,00000000000000000007,1227664369,75635324,41180,H1,027,00424215,476,A,17027,...,1389.984314,1342.309804,1153.725490,929.160784,1219.937255,982.466667,999.949020,1227.074510,1461.415686,1334.643137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,000000000000000008ac,2583464883,392038136,24580,H1,083,01581102,267,A,55083,...,90.000000,155.411765,126.000000,138.129412,128.086275,218.345098,155.243137,213.301961,182.670588,271.819608
954,000000000000000008ad,1372933176,221808928,24580,H1,009,01581064,267,A,55009,...,113.643137,115.831373,134.352941,105.349020,86.278431,176.776471,159.462745,181.658824,218.729412,234.466667
955,000000000000000008ae,2313441606,41883197,43020,H1,115,01581117,267,A,55115,...,138.301961,141.890196,131.901961,144.094118,133.254902,200.929412,186.854902,218.243137,310.749020,291.694118
956,0000000000000000060f,1026784675,17006539,26900,H6,097,00450371,294,C,18097,...,112.784314,88.470588,101.360784,73.113725,83.611765,89.392157,124.564706,90.145098,153.486275,122.980392


In [27]:
# VCount=VCount.drop(columns=['geometry'])
VCount=VCount['GEOID']

In [31]:
# reorder VCount by GEOID
VCount=VCount.sort_values(ascending=True)
#reset the index
VCount=VCount.reset_index(drop=True)
VCount

0      17001
1      17003
2      17005
3      17007
4      17009
       ...  
782    55131
783    55133
784    55135
785    55137
786    55139
Name: GEOID, Length: 787, dtype: object

In [32]:
#save the csv file
VCount.to_csv('../GEE/USID2.csv',index=False)